# Video Sampling Example

This notebook demonstrates how to use the sampling functions to generate videos from text prompts.

In [1]:
import sys
from sample import sample_videos, sample_single_video, prompts
from release_server import load_merge_config, load_all, compile_models

SageAttention loaded successfully
flash attn 2 available False
flash attn 3 available False
sage attn available True
SAGEATTN_AVAILABLE: True
DO_COMPILE False


In [2]:
import torch
import gc
config = load_merge_config("configs/self_forcing_server_14b.yaml")
models = load_all(config)
compile_models(models)
print("✅ Models loaded and ready to use!")
gc.collect()
torch.cuda.empty_cache()


2025-10-20 07:32:28.287 - INFO - Starting model loading...
2025-10-20 07:32:28.288 - DEBUG - Using checkpoint: checkpoints/krea-realtime-video-14b.safetensors
Loading transformer:   0%|          | 0/4 [00:00<?, ?it/s]

2025-10-20 07:32:31.719 - DEBUG - Transformer import took: 3.43s
2025-10-20 07:32:38.530 - DEBUG - Loading transformer state dict from checkpoints/krea-realtime-video-14b.safetensors


Loading WAN model, with name Wan2.1-T2V-14B
timestep shift,  5.0


2025-10-20 07:33:19.605 - DEBUG - Transformer load completed in: 47.89s, total: 51.31s
2025-10-20 07:33:19.608 - DEBUG - Loading transformer took: 51.32s
Loading text encoder:  25%|██▌       | 1/4 [00:51<02:33, 51.32s/it]2025-10-20 07:33:19.610 - DEBUG - Text encoder import took: 0.00s
2025-10-20 07:33:23.142 - DEBUG - Text encoder load completed in: 3.53s, total: 3.53s
2025-10-20 07:33:23.144 - DEBUG - Loading text encoder took: 3.53s
Loading VAE:  50%|█████     | 2/4 [00:55<00:46, 23.21s/it]         2025-10-20 07:33:23.413 - DEBUG - Using demo_utils.vae_block3.VAEEncoderWrapper
2025-10-20 07:33:23.414 - DEBUG - Using demo_utils.vae_block3.VAEDecoderWrapper
2025-10-20 07:33:23.459 - INFO - loading Wan-2.1/Wan2.1-T2V-1.3B/Wan2.1_VAE.pth
2025-10-20 07:33:24.561 - DEBUG - VAE load completed in: 1.15s
2025-10-20 07:33:24.591 - DEBUG - Loading VAE took: 1.18s
Initializing pipeline:  75%|███████▌  | 3/4 [00:56<00:13, 13.27s/it]2025-10-20 07:33:24.597 - DEBUG - Pipeline import took: 0.00s
20

Incompatible _IncompatibleKeys(missing_keys=['mean', 'std'], unexpected_keys=[]) while loading vae decoder
KV inference with 3 frames per block
✅ Models loaded and ready to use!


In [3]:
import sample
from sample import GenerateParams
params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    kv_cache_num_frames=3,
    do_kv_recomp=True,
    num_denoising_steps=4,
    seed=123,
)
prompts = [
    "A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings",
]
results = sample.sample_videos(
    prompts_list=prompts,
    models=models,
    params=params,
    output_dir="outputs",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/1 [00:00<?, ?it/s]


📝 Prompt 1/1: A lone samurai in traditional armor practices kata in a quiet field at dawn, his silhouette framed against a misty horizon. Each sword strike is fluid yet forceful, the blade flashing as it slices through the air with precise arcs and sudden bursts of speed. His feet shift with practiced agility, sending up small sprays of dirt as he pivots, steps, and lunges in a continuous flow of disciplined motion. The camera rotates around the samurai, as he swings
Initializing kv cache with shape:  [1, 9360, 40, 128]
denoising step list:  tensor([1000.0000,  908.8427,  713.9794,    0.0000], device='cuda:0')
🎬 Generating 8 blocks...
Zero reinitialization of kv cache


2025-10-20 07:33:29.468 - DEBUG - Attempting to acquire lock 140403938200144 on /tmp/torchinductor_root/locks/cf5mphb44cdimfvz7xykuxpgy7xslon3smb6gpbg3gac6hy7aovr.lock
2025-10-20 07:33:29.470 - DEBUG - Lock 140403938200144 acquired on /tmp/torchinductor_root/locks/cf5mphb44cdimfvz7xykuxpgy7xslon3smb6gpbg3gac6hy7aovr.lock
2025-10-20 07:33:31.475 - DEBUG - Attempting to release lock 140403938200144 on /tmp/torchinductor_root/locks/cf5mphb44cdimfvz7xykuxpgy7xslon3smb6gpbg3gac6hy7aovr.lock
2025-10-20 07:33:31.477 - DEBUG - Lock 140403938200144 released on /tmp/torchinductor_root/locks/cf5mphb44cdimfvz7xykuxpgy7xslon3smb6gpbg3gac6hy7aovr.lock
2025-10-20 07:33:31.478 - DEBUG - Attempting to acquire lock 140403932033168 on /tmp/torchinductor_root/locks/c6d743rikbxsnp27k5r74gb45fjzusjdfdydltjibkpjqjhl6epp.lock
2025-10-20 07:33:31.479 - DEBUG - Lock 140403932033168 acquired on /tmp/torchinductor_root/locks/c6d743rikbxsnp27k5r74gb45fjzusjdfdydltjibkpjqjhl6epp.lock
2025-10-20 07:33:33.097 - DEBUG

Zero reinitialization of kv cache
Generating block mask
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
Zero reinitialization of kv cache
✅ Generated 90 frames
Sampling took 35.05s


/vast/erwann/realtime-video/.venv/lib/python3.11/site-packages/torchvision/io/_video_deprecation_warning.py:5: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(
Sampling videos: 100%|██████████| 1/1 [00:37<00:00, 37.23s/it]

✅ Video saved to outputs/prompt_000.mp4

🎉 All videos generated successfully!


## Video to Video

In [ ]:

params = GenerateParams(
    prompt="",
    width=832,
    height=480,
    num_blocks=8,
    seed=12345,
    kv_cache_num_frames=3,
    thresh_kv_scale = 1.,
    do_kv_recomp=True,
    num_denoising_steps=4,
    input_video="https://cdn-uploads.huggingface.co/production/uploads/62a2712903bf94c3ac3ae004/UjM1G0hthNGdtItaE-qGc.mp4",
    strength=0.6,
)
prompts = [
    "a white cat knight is riding a horse across a dusty plain. The cat's ears are perked and its snow white fur ripples in the wind as it bounces on the horsea ",
]
results = sample.sample_videos(
    prompts_list=prompts,
    models=models,  # Reuse loaded models
    params=params,
    output_dir="out_v2v",
    save_videos=True,
    fps=16
)

Sampling videos:   0%|          | 0/1 [00:00<?, ?it/s]2025-10-20 07:51:27.862 - DEBUG - Starting new HTTPS connection (1): cdn-uploads.huggingface.co:443



📝 Prompt 1/1: a white cat knight is riding a horse across a dusty plain. The cat's ears are perked and its snow white fur ripples in the wind as it bounces on the horsea 
Zero reinitialization of kv cache
Zero reinitialization of crossattn cache
denoising step list:  tensor([882.3529, 769.2308, 555.5556,   0.0000], device='cuda:0')


2025-10-20 07:51:28.110 - DEBUG - https://cdn-uploads.huggingface.co:443 "GET /production/uploads/62a2712903bf94c3ac3ae004/UjM1G0hthNGdtItaE-qGc.mp4 HTTP/1.1" 200 2024781


rotation 0
read time:  0.12281918525695801
Frame count:  90
